# 1. 사전 세팅

In [ ]:
# Warnings 제거
import warnings
warnings.filterwarnings('ignore')

%cd "/content/drive/MyDrive/데이터 분석/projects/ML_protfoilo/2. 당뇨 환자 재입원 예측 모델링"

# 2. 문제 정의

---

    당뇨병은 세계 성인 인구의 9% 이상에 영향을 미치고 있으며, 점점 더 증가하고 있다.
    당뇨 환자는 지속적인 관찰과 치료가 필요하며, 중증 환자는 더 세심한 관리가 필요하다.
    이를 위해 30일 이내 재입원해야하는 환자를 예측하는 모델을 생성할 것이다.
    그리고 모델 성능 향상을 위해 Error의 분포를 분석해 성능을 향상시킬 것이다.

# 3. 데이터 확인

## 3.1. 데이터 명세

---

| Column | Description |
|:----|:----|
| Features | 환자 입원 기간 내 관련 정보 |
| Class | readmit_30_days |

## 3.2. 데이터 살펴보기

In [ ]:
# openml API를 사용해 당뇨 환자 데이터 읽어오기
from sklearn.datasets import fetch_openml

df, y = fetch_openml(data_id=43874, as_frame=True, return_X_y=True)

In [ ]:
df.info()

In [ ]:
df.sample(5)

## 3.3. 데이터 가공 명세

---

| 명세 | 내용 |
|:----|:----|
| 가공 범위 | <input type="checkbox" checked> Feature(Column) 변형<br> <input type="checkbox">기간 단위 데이터 분석 및 Big Volume 특성을 반영하기 위한 데이터 증강<br> <input type="checkbox" checked>다양한 Table을 참조하기 위한 데이터 분할|
| 가공 상세 | <b>Feature(Column) 변형</b><br><br>- OnehotEncoding Transform을 통한 Feature 추가 생성<br>- 타겟에 직접적으로 관련된 변수 삭제해 모델 영향도 제거<br><br><b>데이터 분할</b><br><br>- 학습/검증을 위해 Train:Test=7:3으로 분할 |
| 가공 결과 | <b>As is</b><br><br>- row: 101766 / column: 34 / table: 1<br><br> <b>To be</b><br><br>- row: 101766 / column: 54(+20) / table: 1  |

# 4. 문제 해결 프로세스 정의

---

▸ 문제

    퇴원한 환자들 중 재입원하는 중증 환자들이 늘어나고 있다.
    
▸ 기대 효과

    재입원 환자들을 예측함으로써 중증 환자 사전 조치와 모니터링을 시행할 수 있다.
    그리고 중증 환자의 생존율이 높아질 수 있다.

▸ 해결 방안

    Binary Classification으로 30일 이내 재입원할 환자들을 분류한다.
    Class Weight가 오류에 미치는 영향을 분석하고, Feature Importance까지도 알아본다.

▸ 성과 측정

    에러/Cohort 분석으로 점진적인 성능 향상 예정

▸ 현업 적용

    모델 성능 하락 시 분석 포인트 탐색

# 5. 데이터 전처리 및 EDA

## 5.1. 데이터 전처리

In [ ]:
# 필요 라이브러리 설치
!pip install lightgbm optuna shap --quiet

In [ ]:
import math
import os
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics

import lightgbm as lgb
import optuna

# from responsibleai import RAIInsights
# from raiwidgets import ResponsibleAIDashboard
# from raiwidgets.cohort import Cohort, CohortFilter, CohortFilterMethods
# Numpy 때문에 호환이 안 된다...
# import shap

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import seaborn as sns

### 5.1.1. Target 유의 변수 제거

In [ ]:
df['readmitted'].value_counts()

| readmitted | count |
|:---:|:----|
| NO | 54864 |
| >30 | 35545 |
| <30 | 11357 |



In [ ]:
df['readmit_binary'].value_counts()

| readmit_binary | count |
|:---:|:----|
| 0 | 54864 |
| 1 | 46902 |

In [ ]:
# 실제 타겟
y.value_counts()

| readmit_30_days | count |
|:---:|:----|
| 0 | 90409 |
| 1 | 11357 |



```
위에서도 알 수 있듯, 2개의 변수는 y와 직접적으로 상관 관계가 보이기 때문에 학습 Feature로 사용할 수 없다.
y와 직접적으로 상관 관계가 있는 변수들을 Feature로 사용할 경우, 모든 경우 100%에 가까운 정확도를 보이게 되며,
이는 정답을 보여주는 것과 동일하기에 정상적인 모델이라고 할 수 없다.
```



### 5.1.2. 인코딩

In [ ]:
# 함수 정의
def transform_data(X_):
    # 변수 제거
    X = X_.drop(['readmitted', 'readmit_binary'], axis=1)

    # 바이너리 인코딩
    bool_cols = X.select_dtypes(include='category').columns.tolist()
    X[bool_cols] = X[bool_cols].astype(str).replace({'True':1, 'False':0})
    print(bool_cols)

    # 원핫 인코딩
    cat_cols = X.select_dtypes(include='object').columns.tolist()
    print(cat_cols)

    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    encoder_np = encoder.fit_transform(X[cat_cols].astype('category'))
    X[encoder.get_feature_names_out(cat_cols)] = encoder_np.astype(int)

    # 원본 열 제거
    X.drop(cat_cols, axis=1, inplace=True)

    return X

In [ ]:
X = transform_data(df)



```
['medicare', 'medicaid', 'had_emergency', 'had_inpatient_days', 'had_outpatient_days']
['race', 'gender', 'age', 'discharge_disposition_id', 'admission_source_id', 'medical_specialty', 'primary_diagnosis', 'max_glu_serum', 'A1Cresult', 'insulin', 'change', 'diabetesMed']
```



In [ ]:
print(f'원본 데이터 Feature 수: {len(df.columns)}')
print(df.columns)



```
원본 데이터 Feature 수: 24
Index(['race', 'gender', 'age', 'discharge_disposition_id',
       'admission_source_id', 'time_in_hospital', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'primary_diagnosis', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'insulin', 'change', 'diabetesMed', 'medicare', 'medicaid',
       'had_emergency', 'had_inpatient_days', 'had_outpatient_days',
       'readmitted', 'readmit_binary'],
      dtype='object')
```



In [ ]:
print(f'변환 데이터 Feature 수: {len(X.columns)}')
print(X.columns)



```
변환 데이터 Feature 수: 54
Index(['time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_diagnoses', 'medicare', 'medicaid',
       'had_emergency', 'had_inpatient_days', 'had_outpatient_days',
       'race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic',
       'race_Other', 'race_Unknown', 'gender_Female', 'gender_Male',
       'gender_Unknown/Invalid', 'age_'30 years or younger'',
       'age_'30-60 years'', 'age_'Over 60 years'',
       'discharge_disposition_id_'Discharged to Home'',
       'discharge_disposition_id_Other', 'admission_source_id_Emergency',
       'admission_source_id_Other', 'admission_source_id_Referral',
       'medical_specialty_Cardiology', 'medical_specialty_Emergency/Trauma',
       'medical_specialty_Family/GeneralPractice',
       'medical_specialty_InternalMedicine', 'medical_specialty_Missing',
       'medical_specialty_Other', 'primary_diagnosis_'Genitourinary Issues'',
       'primary_diagnosis_'Musculoskeletal Issues'',
       'primary_diagnosis_'Respiratory Issues'', 'primary_diagnosis_Diabetes',
       'primary_diagnosis_Other', 'max_glu_serum_>200', 'max_glu_serum_>300',
       'max_glu_serum_None', 'max_glu_serum_Norm', 'A1Cresult_>7',
       'A1Cresult_>8', 'A1Cresult_None', 'A1Cresult_Norm', 'insulin_Down',
       'insulin_No', 'insulin_Steady', 'insulin_Up', 'change_Ch', 'change_No',
       'diabetesMed_No', 'diabetesMed_Yes'],
      dtype='object')
```



## 5.2. 데이터셋 분할

In [ ]:
rand = 42
os.environ['PYTHONHASHSEED'] = str(rand)
np.random.seed(rand)

In [ ]:
label = ['NO or > 30 days', '< 30 days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=rand)
df_test = df.loc[X_test.index]

In [ ]:
# 71236 rows × 54 columns
X_train

In [ ]:
# 30530 rows × 54 columns
X_test

## 5.3. 모델 Evaluation 함수 정의